<a href="https://colab.research.google.com/github/bleakcim/videoGenerator/blob/main/MusicVideo_Complete.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🎵 Music Video Generator - Gerador Automático de Vídeos Musicais

### Crie vídeos completos para suas músicas usando IA!

**O que esse notebook faz:**
1. 🎵 Analisa sua música (intro, verso, refrão, ponte, outro)
2. 🖼️ Gera imagens únicas para cada seção
3. 🎬 Cria vídeos animados de cada imagem
4. 🎞️ Monta tudo sincronizado com a música
5. 🎥 Exporta vídeo pronto para YouTube!

---

## ⚠️ IMPORTANTE: Ativar GPU!

1. Vá em **Runtime → Change runtime type**
2. Selecione **GPU** como Hardware accelerator
3. Clique em **Save**

---

## 📦 Passo 1: Instalar Dependências

Isso vai demorar ~3-5 minutos.

In [ ]:
print("📦 Instalando dependências...\n")

# Instalar bibliotecas principais
!pip install -q diffusers transformers accelerate safetensors xformers
!pip install -q librosa soundfile moviepy imageio imageio-ffmpeg scipy
!pip install -q gradio

print("\n✅ Instalação concluída!")

## 🔍 Passo 2: Verificar GPU

In [ ]:
import torch

if torch.cuda.is_available():
    print("✅ GPU Disponível!")
    print(f"🎮 GPU: {torch.cuda.get_device_name(0)}")
    print(f"💾 VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("❌ GPU NÃO disponível!")
    print("⚠️ Vá em Runtime → Change runtime type → Selecione GPU")

## 🎬 Passo 3: Código do Gerador de Vídeos Musicais

In [ ]:
import os
import json
import numpy as np
from datetime import datetime
from PIL import Image
import torch
from diffusers import StableVideoDiffusionPipeline, StableDiffusionXLPipeline
from diffusers.utils import export_to_video
import librosa
from moviepy.editor import VideoFileClip, AudioFileClip, concatenate_videoclips

class MusicVideoGenerator:
    def __init__(self):
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.dtype = torch.float16 if torch.cuda.is_available() else torch.float32
        self.video_pipeline = None
        self.image_pipeline = None
        print(f"💻 Dispositivo: {self.device.upper()}")

    def load_models(self):
        """Carrega os modelos necessários"""
        try:
            print("\n⏳ Carregando modelos (isso pode demorar 5-10 min na primeira vez)...\n")

            # Modelo para gerar imagens (SDXL)
            print("📥 1/2: Baixando modelo de geração de imagens (SDXL)...")
            self.image_pipeline = StableDiffusionXLPipeline.from_pretrained(
                "stabilityai/stable-diffusion-xl-base-1.0",
                torch_dtype=self.dtype,
                use_safetensors=True,
                variant="fp16" if self.device == "cuda" else None
            )
            self.image_pipeline.to(self.device)

            if self.device == "cuda":
                self.image_pipeline.enable_model_cpu_offload()

            print("✅ Modelo de imagens carregado!\n")

            # Modelo para gerar vídeos (SVD)
            print("📥 2/2: Baixando modelo de geração de vídeos (SVD)...")
            self.video_pipeline = StableVideoDiffusionPipeline.from_pretrained(
                "stabilityai/stable-video-diffusion-img2vid-xt",
                torch_dtype=self.dtype,
                variant="fp16" if self.device == "cuda" else None
            )
            self.video_pipeline.to(self.device)

            if self.device == "cuda":
                self.video_pipeline.enable_model_cpu_offload()

            self.video_pipeline.enable_vae_slicing()

            print("✅ Modelo de vídeos carregado!")
            print("\n🎉 Todos os modelos prontos para uso!\n")
            return True

        except Exception as e:
            print(f"❌ Erro ao carregar modelos: {str(e)}")
            import traceback
            traceback.print_exc()
            return False

    def analyze_music(self, audio_path):
        """Analisa a música e identifica seções"""
        print(f"\n🎵 Analisando música: {audio_path}...")

        # Carregar áudio
        y, sr = librosa.load(audio_path)

        # Detectar batidas e tempo
        tempo, beats = librosa.beat.beat_track(y=y, sr=sr)

        # Converter tempo para float (pode retornar array)
        if isinstance(tempo, np.ndarray):
            tempo = float(tempo[0] if len(tempo) > 0 else tempo)
        else:
            tempo = float(tempo)

        # Detectar mudanças de seção
        boundaries = librosa.segment.agglomerative(
            librosa.feature.mfcc(y=y, sr=sr),
            k=8  # número de seções
        )
        boundary_times = librosa.frames_to_time(boundaries, sr=sr)

        # Analisar energia
        rms = librosa.feature.rms(y=y)[0]

        # Criar estrutura de seções
        sections = []
        for i in range(len(boundary_times) - 1):
            start_time = boundary_times[i]
            end_time = boundary_times[i + 1]
            duration = end_time - start_time

            # Calcular energia média
            start_frame = int(start_time * sr / 512)
            end_frame = int(end_time * sr / 512)
            avg_energy = np.mean(rms[start_frame:end_frame])

            section_type = self._guess_section_type(i, len(boundary_times), avg_energy)

            sections.append({
                'index': i,
                'start': float(start_time),
                'end': float(end_time),
                'duration': float(duration),
                'energy': float(avg_energy),
                'type': section_type
            })

            print(f"  {i+1}. {section_type:12s} | {start_time:6.1f}s - {end_time:6.1f}s | Duração: {duration:5.1f}s | Energia: {avg_energy:.3f}")

        total_duration = float(librosa.get_duration(y=y, sr=sr))

        print(f"\n📊 Total: {len(sections)} seções | {total_duration:.1f}s | {tempo:.0f} BPM\n")

        return {
            'sections': sections,
            'tempo': tempo,
            'total_duration': total_duration
        }

    def _guess_section_type(self, index, total_sections, energy):
        """Identifica o tipo de seção"""
        if index == 0:
            return "Intro"
        elif index == total_sections - 2:
            return "Outro"
        elif energy > 0.15:
            return "Refrão"
        elif energy > 0.10:
            return "Ponte"
        else:
            return "Verso"

    def generate_scene_prompts(self, sections, character, theme, style):
        """Gera prompts visuais para cada seção"""
        print("📝 Gerando roteiro visual...\n")

        base_style = f"{style} style, cinematic, high quality, 4k, detailed"
        char = character if character else "person"

        templates = {
            "Intro": [
                f"{char} in dramatic opening scene, {theme}, mysterious atmosphere, {base_style}",
                f"Close-up of {char}, {theme}, cinematic lighting, {base_style}",
                f"Silhouette of {char}, {theme}, atmospheric, {base_style}"
            ],
            "Verso": [
                f"{char} in storytelling scene, {theme}, narrative moment, {base_style}",
                f"{char} expressing emotion, {theme}, artistic shot, {base_style}",
                f"Medium shot of {char}, {theme}, cinematic composition, {base_style}"
            ],
            "Refrão": [
                f"{char} in energetic scene, {theme}, dynamic action, vibrant colors, {base_style}",
                f"{char} dramatic performance, {theme}, high energy, intense, {base_style}",
                f"Wide shot of {char}, {theme}, spectacular, {base_style}"
            ],
            "Ponte": [
                f"{char} contemplative moment, {theme}, atmospheric, {base_style}",
                f"Artistic shot of {char}, {theme}, creative angle, {base_style}",
                f"{char} transition scene, {theme}, smooth, {base_style}"
            ],
            "Outro": [
                f"{char} closing scene, {theme}, resolution, fading light, {base_style}",
                f"Final shot of {char}, {theme}, memorable ending, {base_style}",
                f"{char} walking away, {theme}, cinematic finale, {base_style}"
            ]
        }

        prompts = []
        for i, section in enumerate(sections):
            section_type = section['type']
            section_templates = templates.get(section_type, templates["Verso"])
            prompt = section_templates[i % len(section_templates)]

            # Ajustar pela energia
            if section['energy'] > 0.15:
                prompt += ", vibrant, energetic, bold colors"
            elif section['energy'] < 0.08:
                prompt += ", calm, peaceful, soft lighting"

            prompts.append({
                'section_index': i,
                'section_type': section_type,
                'prompt': prompt,
                'negative_prompt': "ugly, distorted, low quality, blurry, watermark, text, bad anatomy"
            })

            print(f"  {i+1}. {section_type:12s}: {prompt[:80]}...")

        print()
        return prompts

    def generate_image(self, prompt, negative_prompt, seed=-1):
        """Gera uma imagem"""
        generator = None
        if seed != -1:
            generator = torch.Generator(device=self.device).manual_seed(seed)

        image = self.image_pipeline(
            prompt=prompt,
            negative_prompt=negative_prompt,
            num_inference_steps=30,
            guidance_scale=7.5,
            generator=generator
        ).images[0]

        return image

    def generate_video_from_image(self, image, duration, motion_intensity=127):
        """Gera vídeo de uma imagem"""
        image = image.resize((1024, 576))
        num_frames = max(14, min(50, int(duration * 7)))

        frames = self.video_pipeline(
            image,
            height=576,
            width=1024,
            num_frames=num_frames,
            num_inference_steps=25,
            fps=7,
            motion_bucket_id=motion_intensity,
            decode_chunk_size=4
        ).frames[0]

        return frames

    def create_music_video(
        self,
        audio_path,
        character_description,
        theme,
        style="cinematic",
        seed=-1
    ):
        """Cria o vídeo musical completo"""
        print("\n" + "="*60)
        print("🎬 INICIANDO GERAÇÃO DO VÍDEO MUSICAL")
        print("="*60)

        output_dir = "/content/outputs/music_videos"
        os.makedirs(output_dir, exist_ok=True)
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

        # 1. Analisar música
        music_analysis = self.analyze_music(audio_path)
        sections = music_analysis['sections']

        # 2. Gerar prompts
        scene_prompts = self.generate_scene_prompts(
            sections, character_description, theme, style
        )

        # 3. Gerar imagens e vídeos
        print("🎬 Gerando cenas...\n")
        video_clips = []

        for i, (section, scene) in enumerate(zip(sections, scene_prompts)):
            print(f"[{i+1}/{len(sections)}] 🎨 Gerando imagem para {section['type']}...")

            # Gerar imagem
            image = self.generate_image(
                scene['prompt'],
                scene['negative_prompt'],
                seed if seed != -1 else -1
            )

            # Salvar imagem
            image_path = f"{output_dir}/scene_{timestamp}_{i:02d}.png"
            image.save(image_path)
            print(f"         ✅ Imagem salva: {image_path}")

            # Gerar vídeo
            print(f"         🎞️  Gerando vídeo ({section['duration']:.1f}s)...")
            motion_intensity = int(100 + (section['energy'] * 500))
            motion_intensity = min(200, max(80, motion_intensity))

            frames = self.generate_video_from_image(
                image, section['duration'], motion_intensity
            )

            # Salvar vídeo temporário
            temp_video = f"{output_dir}/temp_{timestamp}_{i:02d}.mp4"
            export_to_video(frames, temp_video, fps=7)

            # Ajustar duração
            clip = VideoFileClip(temp_video)
            if clip.duration < section['duration']:
                clip = clip.loop(duration=section['duration'])
            else:
                clip = clip.subclip(0, section['duration'])

            video_clips.append(clip)
            print(f"         ✅ Vídeo gerado!\n")

        # 4. Montar vídeo final
        print("🎞️  Montando vídeo final...")
        final_video = concatenate_videoclips(video_clips, method="compose")

        # 5. Adicionar áudio
        print("🔊 Adicionando áudio...")
        audio = AudioFileClip(audio_path)

        if final_video.duration > audio.duration:
            final_video = final_video.subclip(0, audio.duration)
        elif final_video.duration < audio.duration:
            audio = audio.subclip(0, final_video.duration)

        final_video = final_video.set_audio(audio)

        # 6. Salvar
        output_path = f"{output_dir}/music_video_{timestamp}.mp4"
        print(f"💾 Salvando vídeo final em {output_path}...")

        final_video.write_videofile(
            output_path,
            codec='libx264',
            audio_codec='aac',
            fps=24,
            preset='medium',
            threads=4
        )

        # Limpar
        for clip in video_clips:
            clip.close()
        final_video.close()
        audio.close()

        print("\n" + "="*60)
        print("🎉 VÍDEO MUSICAL CRIADO COM SUCESSO!")
        print("="*60)
        print(f"\n📁 Arquivo: {output_path}")
        print(f"⏱️  Duração: {music_analysis['total_duration']:.1f}s")
        print(f"🎬 Cenas: {len(sections)}")
        print(f"🎵 BPM: {music_analysis['tempo']:.0f}")
        print()

        return output_path

# Criar instância do gerador
generator = MusicVideoGenerator()

print("✅ Classe MusicVideoGenerator carregada!")

## 🔄 Passo 4: Carregar os Modelos de IA

**⚠️ IMPORTANTE:** Isso vai baixar ~14 GB de modelos na primeira execução (5-10 min)

In [ ]:
# Carregar modelos
success = generator.load_models()

if success:
    print("🚀 Tudo pronto para gerar vídeos!")
else:
    print("❌ Erro ao carregar modelos. Verifique os erros acima.")

## 🎵 Passo 5: Upload da sua Música

Faça upload do arquivo de áudio (MP3, WAV, etc)

In [ ]:
from google.colab import files

print("📤 Faça upload da sua música:")
uploaded = files.upload()

# Pegar o nome do arquivo
audio_file = list(uploaded.keys())[0]
print(f"\n✅ Arquivo carregado: {audio_file}")

## 🎨 Passo 6: Configurar o Vídeo

In [ ]:
# ===== CONFIGURE AQUI =====

# Descrição do personagem principal
# Exemplos:
# - "young woman with long black hair, wearing elegant dress"
# - "muscular man with tattoos, wearing leather jacket"
# - "anime girl with blue hair and magical powers"
CHARACTER = "beautiful person with flowing hair"

# Tema/Ambiente do vídeo
# Exemplos:
# - "cyberpunk city with neon lights"
# - "enchanted forest with mystical atmosphere"
# - "futuristic space station"
# - "urban streets at sunset"
THEME = "urban landscape at golden hour"

# Estilo visual
# Opções: cinematic, anime, photorealistic, oil painting, digital art, watercolor, 3D render
STYLE = "cinematic"

# Seed (use -1 para aleatório, ou número fixo para resultados reproduzíveis)
SEED = -1

print("✅ Configuração definida:")
print(f"  👤 Personagem: {CHARACTER}")
print(f"  🌍 Tema: {THEME}")
print(f"  🎨 Estilo: {STYLE}")
print(f"  🎲 Seed: {SEED}")

## 🚀 Passo 7: GERAR O VÍDEO MUSICAL!

**⏱️ Tempo estimado:**
- Música de 3 min com GPU T4: ~15-20 minutos
- Cada cena leva ~2-3 minutos

**☕ Pegue um café enquanto a IA trabalha!**

In [ ]:
import time

start_time = time.time()

# GERAR O VÍDEO MUSICAL!
output_video_path = generator.create_music_video(
    audio_path=audio_file,
    character_description=CHARACTER,
    theme=THEME,
    style=STYLE,
    seed=SEED
)

elapsed_time = time.time() - start_time
print(f"\n⏱️  Tempo total: {elapsed_time/60:.1f} minutos")
print(f"\n🎉 Seu vídeo musical está pronto!")
print(f"📁 Localização: {output_video_path}")

## 🎥 Passo 8: Visualizar o Vídeo

In [ ]:
from IPython.display import Video

# Mostrar o vídeo
Video(output_video_path, width=800)

## 📥 Passo 9: Baixar o Vídeo

In [ ]:
from google.colab import files

# Baixar o vídeo final
print("📥 Baixando vídeo...")
files.download(output_video_path)

print("✅ Download iniciado! O arquivo será salvo na sua pasta de Downloads.")

## 📦 (Opcional) Baixar Todas as Imagens Geradas

In [ ]:
import glob
from google.colab import files

# Encontrar todas as imagens
images = glob.glob("/content/outputs/music_videos/scene_*.png")

print(f"📸 Encontradas {len(images)} imagens\n")

# Baixar cada imagem
for img in sorted(images):
    print(f"📥 {os.path.basename(img)}")
    files.download(img)

print("\n✅ Todas as imagens foram baixadas!")

---

## 💡 Dicas e Exemplos

### 🎤 Exemplos de Configurações

#### 1. Vídeo Pop Energético:
```python
CHARACTER = "young woman with colorful hair, wearing trendy outfit"
THEME = "vibrant city lights, neon signs, urban nightlife"
STYLE = "cinematic"
```

#### 2. Vídeo Anime/J-Pop:
```python
CHARACTER = "anime girl with pink hair and magical girl outfit"
THEME = "fantasy world with floating islands and cherry blossoms"
STYLE = "anime"
```

#### 3. Vídeo Rock/Metal:
```python
CHARACTER = "rock musician with long hair and leather jacket"
THEME = "industrial wasteland, apocalyptic atmosphere, dramatic sky"
STYLE = "cinematic"
```

#### 4. Vídeo Eletrônica/EDM:
```python
CHARACTER = "silhouette of DJ with glowing elements"
THEME = "futuristic club, laser lights, holographic effects, neon colors"
STYLE = "digital art"
```

#### 5. Vídeo Lo-fi/Chill:
```python
CHARACTER = "person studying at desk with headphones"
THEME = "cozy room, rainy window, warm lighting, peaceful atmosphere"
STYLE = "watercolor"
```

### 🎨 Dicas de Personagens:
- Seja **específico**: cor de cabelo, roupas, acessórios
- Mantenha **consistente** em todo vídeo
- Adicione características marcantes

### 🌍 Dicas de Tema:
- Combine local + atmosfera + iluminação
- Use adjetivos visuais (neon, mystical, dramatic)
- Pense na estética que combina com sua música

### ⚡ Performance:
- **GPU T4 (Colab)**: ~2-3 min por cena
- **Música de 3 min**: ~8 cenas = 15-20 min total
- **Música de 5 min**: ~12 cenas = 25-30 min total

### 🎬 Para YouTube:
O vídeo é gerado em:
- **Resolução**: 1024x576 (16:9)
- **FPS**: 24
- **Codec**: H.264
- Pronto para upload direto!

---

## 🆘 Problemas?

### ❌ "Out of memory":
- Certifique-se que GPU está ativada
- Reinicie o runtime: Runtime → Restart runtime
- Música muito longa? Tente uma mais curta primeiro

### ⏰ "Tempo esgotado":
- Colab grátis tem 12h de limite
- Salve o vídeo antes do tempo acabar

### 🖼️ Imagens não ficaram boas:
- Melhore a descrição do CHARACTER
- Seja mais específico no THEME
- Tente um SEED diferente
- Experimente outro STYLE

---

**🎉 Desenvolvido com ❤️ usando Stable Diffusion XL + Stable Video Diffusion**